In [33]:
from pyannote.database import registry
from pyannote.audio.tasks import SpeakerDiarization, Segmentation
from google.cloud import secretmanager
from pyannote.audio import Model
from pyannote.audio import Pipeline
from pyannote.audio import tasks
import os
from pyannote.database import get_protocol, FileFinder

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """

    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(name=name)

    # Return the decoded payload.
    return response.payload.data.decode('UTF-8')

HF_KEY = access_secret_version(1076126751560, "huggingface-api-key", "latest")



E0805 16:30:33.035462000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035482000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035488000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035493000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035498000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035502000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035506000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Illegal keepalive throttling value 
E0805 16:30:33.035510000 140704415147776 client_channel.cc:667]        chand=0x7ff4280f9aa0: Ille

In [73]:
os.environ["PYANNOTE_DATABASE_CONFIG"] = "/Users/StephenWitkowski/Projects/broadcastify-transcription/data/broadcastify_database.yaml"
registry.load_database("/Users/StephenWitkowski/Projects/broadcastify-transcription/data/broadcastify_database.yaml")

dataset = get_protocol("MyDatabase.SpeakerDiarization.MyProtocol", {"audio": FileFinder()})

pretrained_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True) 

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1. Bad things might happen unless you revert torch to 1.x.


In [76]:
# this takes approximately 2min to run on Google Colab GPU
from pyannote.metrics.diarization import DiarizationErrorRate
metric = DiarizationErrorRate()

for file in dataset.test():
    # apply pretrained pipeline
    file["pretrained pipeline"] = pretrained_pipeline(file)

    # evaluate its performance
    metric(file["annotation"], file["pretrained pipeline"], uem=file["annotated"])

    break

print(f"The pretrained pipeline reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {dataset.name} test set.")

In [32]:
Segmentation?

Object `Segmentation` not found.


In [36]:
task = Segmentation(
    dataset, 
    duration=2,
    max_speakers_per_chunk = 2,
    max_speakers_per_frame = 1,
    batch_size=32,
    num_workers=2, 
    loss="bce")
model.task = task
model.setup(stage="fit")

In [37]:
# this takes approximately 15min to run on Google Colab GPU
from types import MethodType
from torch.optim import Adam
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichProgressBar,
)



# we use Adam optimizer with 1e-4 learning rate
def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-4)

model.configure_optimizers = MethodType(configure_optimizers, model)

# we monitor diarization error rate on the validation set
# and use to keep the best checkpoint and stop early
monitor, direction = task.val_monitor
checkpoint = ModelCheckpoint(
    monitor=monitor,
    mode=direction,
    save_top_k=1,
    every_n_epochs=1,
    save_last=False,
    save_weights_only=False,
    filename="{epoch}",
    verbose=False,
)

early_stopping = EarlyStopping(
    monitor=monitor,
    mode=direction,
    min_delta=0.0,
    patience=10,
    strict=True,
    verbose=False,
)

callbacks = [RichProgressBar(), checkpoint, early_stopping]

# we train for at most 20 epochs (might be shorter in case of early stopping)
from pytorch_lightning import Trainer
trainer = Trainer(accelerator="cpu", 
                    logger=False,
                  callbacks=callbacks, 
                  max_epochs=1,
                  gradient_clip_val=0.5)
trainer.fit(model)
     

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Name              ┃ Type             ┃ Params ┃      In sizes ┃                                   Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0 │ sincnet           │ SincNet          │ 42.6 K │ [1, 1, 32000] │                                [1, 60, 115] │
│ 1 │ lstm              │ LSTM             │  1.4 M │  [1, 115, 60] │ [[1, 115, 256], [[8, 1, 128], [8, 1, 128]]] │
│ 2 │ linear            │ ModuleList       │ 49.4 K │             ? │                                           ? │
│ 3 │ classifier        │ Linear           │    387 │ [1, 115, 128] │                                 [1, 115, 3] │
│ 4 │ activation        │ LogSoftmax       │      0 │   [1, 115, 3] │                                 [1, 115, 3] │
│ 5 │ powerset          │ Powerset         │      0 │             ? │                                           ? │
│ 6 │ validation_metric │ MetricCollection │      0 │             ? │                                           ? │
└───┴───────────────────┴──────────────────┴────────┴───────────────┴─────────────────────────────────────────────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 5

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.


In [59]:
finetuned_model = checkpoint.best_model_path

In [61]:
dir(checkpoint)

['CHECKPOINT_JOIN_CHAR',
 'CHECKPOINT_NAME_LAST',
 'FILE_EXTENSION',
 'STARTING_VERSION',
 '_ModelCheckpoint__init_ckpt_dir',
 '_ModelCheckpoint__init_monitor_mode',
 '_ModelCheckpoint__init_triggers',
 '_ModelCheckpoint__resolve_ckpt_dir',
 '_ModelCheckpoint__validate_init_configuration',
 '_ModelCheckpoint__warn_if_dir_not_empty',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_every_n_epochs',
 '_every_n_train_steps',
 '_find_last_checkpoints',
 '_format_checkpoint_name',
 '_fs',
 '_generate_state_key',
 '_get_metric_interpolated_filepath_name',
 '_last_global_step_saved',
 '_last_time_checked',
 '_legacy_state_key',
 '_monitor_candidates',
 '_rem

In [58]:
dir(finetuned_model)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'stri

In [54]:
from pyannote.audio import Pipeline
pretrained_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True)

pretrained_hyperparameters = pretrained_pipeline.parameters(instantiated=True)
pretrained_hyperparameters


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1. Bad things might happen unless you revert torch to 1.x.


{'segmentation': {'min_duration_off': 0.5817029604921046,
  'threshold': 0.4442333667381752},
 'clustering': {'method': 'centroid',
  'min_cluster_size': 15,
  'threshold': 0.7153814381597874}}

In [53]:
# this takes approximately 5min to run on Google Colab GPU
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.pipeline import Optimizer

pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    clustering="OracleClustering",  
)
# as reported in the technical report, min_duration_off can safely be set to 0.0
pipeline.freeze(pretrained_hyperparameters)
pipeline.freeze({"segmentation": {"min_duration_off": 0.0}})


ValueError: parameter 'threshold' does not exist

In [51]:
pipeline.parameters()

{'segmentation': {'min_duration_off': <pyannote.pipeline.parameter.Frozen at 0x160a98450>},
 'clustering': {}}

In [65]:
Optimizer?

Init signature:
Optimizer(
    pipeline: pyannote.pipeline.pipeline.Pipeline,
    db: Optional[pathlib.Path] = None,
    study_name: Optional[str] = None,
    sampler: Union[str, optuna.samplers._base.BaseSampler, NoneType] = None,
    pruner: Union[str, optuna.pruners._base.BasePruner, NoneType] = None,
)
Docstring:     
Pipeline optimizer

Parameters
----------
pipeline : `Pipeline`
    Pipeline.
db : `Path`, optional
    Path to iteration database on disk.
study_name : `str`, optional
    Name of study. In case it already exists, study will continue from
    there. # TODO -- generate this automatically
sampler : `str` or sampler instance, optional
    Algorithm for value suggestion. Must be one of "RandomSampler" or
    "TPESampler", or a sampler instance. Defaults to "TPESampler".
pruner : `str` or pruner instance, optional
    Algorithm for early pruning of trials. Must be one of "MedianPruner" or
    "SuccessiveHalvingPruner", or a pruner instance.
    Defaults to no pruning.
Fil

In [62]:

optimizer = Optimizer(pipeline)
dev_set = list(dataset.development())

iterations = optimizer.tune_iter(dev_set, show_progress=False)
best_loss = 1.0
for i, iteration in enumerate(iterations):
    print(f"Best segmentation threshold so far: {iteration}")
    if i > 20: break   # 50 iterations should give slightly better results

Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': {'segmentation': {'min_duration_off': 0.5817029604921046}, 'clustering': {}}}
Best segmentation threshold so far: {'loss': 0.40268080172031917, 'params': 

In [21]:
iteration

{'loss': 0.4876474198398448,
 'params': {'segmentation': {'min_duration_off': 0.0}, 'clustering': {}}}

In [30]:
params = pretrained_pipeline.parameters(instantiated=True)


In [31]:
# this takes approximately 5min to run on Google Colab GPU
pipeline = SpeakerDiarization(
    segmentation=finetuned_model,
    embedding=pretrained_pipeline.embedding,
    embedding_exclude_overlap=pretrained_pipeline.embedding_exclude_overlap,
    clustering=pretrained_pipeline.klustering,
)

pipeline.freeze(params)


ValueError: parameter 'threshold' does not exist

In [25]:

# 4. apply pretrained pipeline
diarization = pipeline("/Users/StephenWitkowski/Projects/broadcastify-transcription/data/audio/input_chunk_102.wav")

# 5. print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")

RuntimeError: A pipeline must be instantiated with `pipeline.instantiate(parameters)` before it can be applied.

In [66]:
model = Model.from_pretrained(finetuned_model, use_auth_token=True)

In [71]:
model("/Users/StephenWitkowski/Projects/broadcastify-transcription/data/audio/input_chunk_13.wav")

AttributeError: 'str' object has no attribute 'dim'

In [68]:
dir(model)

['CHECKPOINT_HYPER_PARAMS_KEY',
 'CHECKPOINT_HYPER_PARAMS_NAME',
 'CHECKPOINT_HYPER_PARAMS_TYPE',
 'LINEAR_DEFAULTS',
 'LSTM_DEFAULTS',
 'SINCNET_DEFAULTS',
 'T_destination',
 '_DeviceDtypeModuleMixin__update_properties',
 '_LightningModule__check_allowed',
 '_LightningModule__check_not_nested',
 '_LightningModule__to_tensor',
 '_Model__by_name',
 '_Model__example_input_array',
 '_Model__up_to',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__jit_unused_properties__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_apply_batch_transfer_handler',
 '_automatic_optimization',
 '_backward_hooks',
 '_backward_pre_hooks',
 

In [70]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(model,
                                    use_auth_token="ACCESS_TOKEN_GOES_HERE")


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'PyanNet(
  (sincnet): SincNet(
    (wav_norm1d): InstanceNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    (conv1d): ModuleList(
      (0): Encoder(
        (filterbank): ParamSincFB()
      )
      (1): Conv1d(80, 60, kernel_size=(5,), stride=(1,))
      (2): Conv1d(60, 60, kernel_size=(5,), stride=(1,))
    )
    (pool1d): ModuleList(
      (0-2): 3 x MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
    (norm1d): ModuleList(
      (0): InstanceNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (1-2): 2 x InstanceNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
  )
  (lstm): LSTM(60, 128, num_layers=4, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): ModuleList(
    (0): Linear(in_feature

In [72]:
dir(Pipeline)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_flatten',
 '_flattened_parameters',
 '_unflatten',
 'classes',
 'default_parameters',
 'dump_params',
 'freeze',
 'from_pretrained',
 'get_direction',
 'get_metric',
 'initialize',
 'instantiate',
 'instantiated',
 'load_params',
 'loss',
 'parameters',
 'setup_hook',
 'to',
 'training',
 'write',
 'write_format',
 'write_rttm',
 'write_txt']